# Cloud-Aware Protocol

> Protocol for plugins that use cloud or remote computing resources

In [ ]:
#| default_exp protocols.cloud_aware

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Protocol, runtime_checkable, Optional

from cjm_fasthtml_plugins.core.metadata import RemoteResourceInfo

## Cloud-Aware Plugin Protocol

This protocol defines the interface for plugins that execute on cloud or remote resources.

Use cases:
- Cloud GPU rental (Lambda Labs, RunPod, Vast.ai)
- Cloud VMs (AWS EC2, GCP Compute)
- Cloud containers (AWS ECS, Cloud Run)
- Remote SSH execution

Plugins implementing this protocol enable:
- Cost tracking and estimation
- Resource provisioning and termination
- Status monitoring
- Budget enforcement

In [ ]:
#| export
@runtime_checkable
class CloudAwarePlugin(Protocol):
    """Protocol for plugins that use cloud/remote resources."""
    
    def get_remote_resource_info(self) -> Optional[RemoteResourceInfo]:  # RemoteResourceInfo if resources are provisioned, None otherwise
        """Get information about remote/cloud resources."""
        ...
    
    def provision_remote_resource(self, **config) -> RemoteResourceInfo:  # RemoteResourceInfo with details about provisioned resource
        """Provision cloud resources (VM, container, etc.)."""
        ...
    
    def check_remote_resource_status(self) -> str:  # Status string (e.g., 'running', 'stopped', 'provisioning')
        """Check status of remote resource."""
        ...
    
    def terminate_remote_resource(self) -> bool:  # True if termination succeeded
        """Terminate/stop cloud resources to avoid costs."""
        ...
    
    def estimate_cost(self, 
                     duration_hours: float  # Estimated runtime in hours
                    ) -> float:  # Estimated cost in USD
        """Estimate cost for running this duration."""
        ...

## Example Implementation

Here's a mock cloud-based plugin:

In [ ]:
# Example: Mock cloud plugin
from cjm_fasthtml_plugins.core.execution_mode import CloudProviderType

class MockCloudPlugin:
    """Mock plugin that uses cloud resources."""
    
    def __init__(self):
        self.remote_resource: Optional[RemoteResourceInfo] = None
    
    # Implement CloudAwarePlugin protocol
    
    def get_remote_resource_info(self) -> Optional[RemoteResourceInfo]:
        return self.remote_resource
    
    def provision_remote_resource(self, **config) -> RemoteResourceInfo:
        print("Provisioning cloud GPU instance...")
        
        # Mock provisioning
        self.remote_resource = RemoteResourceInfo(
            provider=CloudProviderType.LAMBDA_LABS,
            instance_id="0x1a2b3c4d",
            resource_type="4x_A100",
            gpu_count=4,
            gpu_type="A100",
            status="running",
            estimated_cost_per_hour=4.40,
            ssh_host="54.123.45.67",
            region="us-west-1"
        )
        
        print(f"Provisioned: {self.remote_resource.resource_type}")
        return self.remote_resource
    
    def check_remote_resource_status(self) -> str:
        if not self.remote_resource:
            return "not_provisioned"
        return self.remote_resource.status
    
    def terminate_remote_resource(self) -> bool:
        if self.remote_resource:
            print(f"Terminating instance: {self.remote_resource.instance_id}")
            self.remote_resource.status = "terminated"
            return True
        return False
    
    def estimate_cost(self, duration_hours: float) -> float:
        if not self.remote_resource:
            return 0.0
        hourly = self.remote_resource.estimated_cost_per_hour or 0.0
        return duration_hours * hourly

In [ ]:
# Test the mock plugin
plugin = MockCloudPlugin()

# Check it implements the protocol
print(f"Implements protocol: {isinstance(plugin, CloudAwarePlugin)}")

# Check initial state
print(f"\nInitial status: {plugin.check_remote_resource_status()}")
print(f"Resource info: {plugin.get_remote_resource_info()}")

# Provision resources
resource = plugin.provision_remote_resource()
print(f"\nAfter provisioning:")
print(f"  Status: {plugin.check_remote_resource_status()}")
print(f"  Provider: {resource.provider.value}")
print(f"  GPUs: {resource.gpu_count}x {resource.gpu_type}")
print(f"  Cost: ${resource.estimated_cost_per_hour}/hour")

# Estimate costs
cost_1hr = plugin.estimate_cost(1.0)
cost_8hr = plugin.estimate_cost(8.0)
print(f"\nCost estimates:")
print(f"  1 hour: ${cost_1hr:.2f}")
print(f"  8 hours: ${cost_8hr:.2f}")

# Terminate
success = plugin.terminate_remote_resource()
print(f"\nTermination successful: {success}")
print(f"Final status: {plugin.check_remote_resource_status()}")

Implements protocol: True

Initial status: not_provisioned
Resource info: None
Provisioning cloud GPU instance...
Provisioned: 4x_A100

After provisioning:
  Status: running
  Provider: lambda_labs
  GPUs: 4x A100
  Cost: $4.4/hour

Cost estimates:
  1 hour: $4.40
  8 hours: $35.20
Terminating instance: 0x1a2b3c4d

Termination successful: True
Final status: terminated


## Helper Functions

Utilities for working with cloud-aware plugins.

In [ ]:
#| export
from typing import Any, List

def is_cloud_aware(plugin: Any  # Plugin instance to check
                  ) -> bool:  # True if plugin implements the protocol
    """Check if a plugin implements the CloudAwarePlugin protocol."""
    return isinstance(plugin, CloudAwarePlugin)

def has_active_cloud_resources(plugin: Any  # Plugin instance
                               ) -> bool:  # True if plugin has running cloud resources
    """Check if plugin has active cloud resources."""
    if not is_cloud_aware(plugin):
        return False
    
    resource_info = plugin.get_remote_resource_info()
    if not resource_info:
        return False
    
    return resource_info.status == "running"

def get_total_estimated_cost(plugins: List[Any],  # List of plugin instances
                            duration_hours: float = 1.0  # Duration to estimate for
                           ) -> float:  # Total estimated cost in USD
    """Get total estimated cost for multiple plugins."""
    total = 0.0
    for plugin in plugins:
        if is_cloud_aware(plugin) and has_active_cloud_resources(plugin):
            total += plugin.estimate_cost(duration_hours)
    return total

In [ ]:
# Test helper functions
plugin1 = MockCloudPlugin()
plugin1.provision_remote_resource()

plugin2 = MockCloudPlugin()
plugin2.provision_remote_resource()

print(f"Plugin 1 is cloud aware: {is_cloud_aware(plugin1)}")
print(f"Plugin 1 has active resources: {has_active_cloud_resources(plugin1)}")

# Test with multiple plugins
plugins = [plugin1, plugin2, {"not": "a plugin"}]
total_cost_1hr = get_total_estimated_cost(plugins, 1.0)
total_cost_24hr = get_total_estimated_cost(plugins, 24.0)

print(f"\nTotal cost for 2 plugins:")
print(f"  1 hour: ${total_cost_1hr:.2f}")
print(f"  24 hours: ${total_cost_24hr:.2f}")

# Terminate resources
plugin1.terminate_remote_resource()
plugin2.terminate_remote_resource()

print(f"\nAfter termination:")
print(f"Plugin 1 has active resources: {has_active_cloud_resources(plugin1)}")

Provisioning cloud GPU instance...
Provisioned: 4x_A100
Provisioning cloud GPU instance...
Provisioned: 4x_A100
Plugin 1 is cloud aware: True
Plugin 1 has active resources: True

Total cost for 2 plugins:
  1 hour: $8.80
  24 hours: $211.20
Terminating instance: 0x1a2b3c4d
Terminating instance: 0x1a2b3c4d

After termination:
Plugin 1 has active resources: False


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()